In [109]:
import pickle
import numpy as np
import tensorflow as tf
from keyword_spotting.model import models
import netron

In [118]:
model_name = 'vit2010/cnn_visiontransformer_23_25_34_Jun_13_2021'
#model_name = 'res3/cnn_residual2_19_04_14_Jun_13_2021'
#model_name='res6/cnn_residual2_09_20_43_Jun_19_2021'
#model_name='res9/cnn_residual2_11_28_12_Jun_19_2021'

In [21]:
netron.start(f'/Users/emi/resultadosHDA/{model_name}.h5')

Serving '/Users/emi/resultadosHDA/vit2010/cnn_visiontransformer_23_25_34_Jun_13_2021.h5' at http://localhost:8080


('localhost', 8080)

In [119]:
configFILE=f'/Users/emi/resultadosHDA/{model_name}_results.pkl'
modelFILE = f'/Users/emi/resultadosHDA/{model_name}'
with open(configFILE, 'rb') as f:
    config, time, predictions = pickle.load(f)

In [120]:
config

{'model': {'name': 'cnn_visiontransformer',
  'params': {'learning_rate': 0.001,
   'n_dense_layer': 1,
   'num_heads': 3,
   'transformer_layers': 4,
   'projection_dim': 16,
   'patch_x': 20,
   'patch_y': 10,
   'dense_dropout': 0.2},
  'windowed': False},
 'train': {'batch_size': 64, 'epochs': 27, 'reduce_on_plateau': True}}

In [90]:
#config['model']['params']['n_residuals']=6
config['model']['params']['patch_x']=10
config['model']['params']['patch_y']=8

In [117]:
# hour
time/60/60

time

9814.142082691193

In [113]:
input_shape = [100, 40]
number_of_classes=12

model = models[config["model"]["name"]](input_shape, number_of_classes, **config["model"]["params"])
print(model.summary())

Model: "model_20"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           [(None, 100, 40)]    0                                            
__________________________________________________________________________________________________
lambda_20 (Lambda)              (None, 100, 40, 1)   0           input_21[0][0]                   
__________________________________________________________________________________________________
conv2d_77 (Conv2D)              (None, 98, 38, 19)   190         lambda_20[0][0]                  
__________________________________________________________________________________________________
average_pooling2d_5 (AveragePoo (None, 49, 19, 19)   0           conv2d_77[0][0]                  
___________________________________________________________________________________________

In [100]:
model.save('/Users/emi/resultadosHDA/vit2010/cnn_visiontransformer_23_25_34_Jun_13_2021.h5')

In [101]:
# Restore the weights
model.load_weights(f'{modelFILE}')

In [114]:
import pickle
from pathlib import Path
configFILE=f'/Users/emi/resultadosHDA/{model_name}_results.pkl'
with open(configFILE, 'rb') as f:
    results = pickle.load(f)

In [115]:
from keyword_spotting.predictions import evaluate_predictions,labels
from sklearn.metrics import accuracy_score, f1_score

In [116]:
y_true, y_pred = evaluate_predictions(results[2], Path('/Users/emi/unipd/hda/dataset/'))
accuracy_score(y_true, y_pred)

0.8629893238434164

In [35]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 8))

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=labels)
disp.plot(ax=ax) 

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
disp